In [ ]:
import os
import time
import requests

API_TOKEN = "eoimbf8yx8u07kje3scrmyhhjro9fobhs30ysb9q"  # or paste your key as a string
BASE_URL = "https://stocknewsapi.com/api/v1"  # ticker news endpoint

from tqdm import tqdm
import requests, time
from datetime import datetime

def fetch_news_from_2021_07_20(limit_per_page=20, sleep_sec=0.25, end_date=None, start_date=None, ticker="NVDA"):
    """
    Fetches historical news starting on 2021-07-20.
    If end_date is provided, it must be a string in MMDDYYYY format, e.g., "10162025".
    Returns a list of article dicts.
    """
    if not API_TOKEN:
        raise RuntimeError("Set STOCKNEWSAPI_TOKEN env var or hardcode API_TOKEN")

    date_param = start_date if end_date is None else f"{start_date}-{end_date}"

    page = 1
    all_news = []
    unique_dates = set()

    # Initialize tqdm without known total
    pbar = tqdm(desc=f"Fetching {ticker} news", unit="date(s)")

    while True:
        if page == 10:  # your original cutoff
            break

        params = {
            "tickers": ticker,
            "items": limit_per_page,
            "page": page,
            "date": date_param,
            "token": API_TOKEN,
        }
        resp = requests.get(BASE_URL, params=params, timeout=30)
        resp.raise_for_status()
        data = resp.json()

        articles = data.get("data", [])
        if not articles:
            break

        all_news.extend(articles)

        # Extract pub dates from articles (assuming "date" field exists)
        for a in articles:
            pubdate = a.get("date")  # e.g. "2023-10-03T12:30:00Z"
            if pubdate:
                day = pubdate.split("T")[0]  # just YYYY-MM-DD
                if day not in unique_dates:
                    unique_dates.add(day)
                    pbar.update(1)  # one more unique date discovered

        page += 1
        time.sleep(sleep_sec)

    pbar.close()
    print(f"\nTotal unique dates collected: {len(unique_dates)}")

    return all_news


# # # Example usage
# # if __name__ == "__main__":
# #     news = fetch_nvda_news_from_2021_07_20(end_date="08292025", start_date="07292025")  # set end_date like "10162025" if you want a cutoff
# #     print(f"Total articles: {len(news)}")
# #     # Peek
# #     for a in news[:5]:
# #         print(a.get("date"), a.get("title"))


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
ticker1 = "NVDA"
def fetch_news_monthly_backward(start_date_str="10052025", end_date_str="10052025", stop_date_str="07052021"):
    """
    Runs fetch_news_from_2021_07_20 in backward monthly windows.
    start_date_str and end_date_str should be in MMDDYYYY format.
    stop_date_str is the lower bound when the loop stops.
    Returns a single concatenated dataframe.
    """
    parent_df = pd.DataFrame()

    # Convert strings to datetime objects
    current_start = datetime.strptime(start_date_str, "%m%d%Y")
    current_end   = datetime.strptime(end_date_str, "%m%d%Y")
    stop_date     = datetime.strptime(stop_date_str, "%m%d%Y")

    while current_start >= stop_date:
        # Call your existing function EXACTLY as-is
        data = fetch_news_from_2021_07_20(
            start_date=current_start.strftime("%m%d%Y"),
            end_date=current_end.strftime("%m%d%Y"),
            ticker=ticker1
        )

        # Convert to df and append
        df = pd.DataFrame(data)
        parent_df = pd.concat([parent_df, df], ignore_index=True)

        print(f"Collected {len(df)} rows from {current_start.strftime('%m/%d/%Y')} to {current_end.strftime('%m/%d/%Y')}")

        # Move one month back (30-day window)
        current_end = current_start
        current_start = current_start - timedelta(days=9)

    return parent_df

In [ ]:
df_all = fetch_news_monthly_backward(
    start_date_str="10052025",
    end_date_str="10052025",
    stop_date_str="06252021"
)

Fetching NVDA news: 7date(s) [00:01,  6.41date(s)/s]



Total unique dates collected: 7
Collected 7 rows from 10/05/2025 to 10/05/2025


Fetching NVDA news: 109date(s) [00:06, 16.71date(s)/s]



Total unique dates collected: 109
Collected 141 rows from 09/26/2025 to 10/05/2025


Fetching NVDA news: 99date(s) [00:06, 14.76date(s)/s]


...


Total unique dates collected: 22
Collected 41 rows from 07/19/2021 to 07/28/2021


Fetching NVDA news: 23date(s) [00:01, 12.68date(s)/s]



Total unique dates collected: 23
Collected 28 rows from 07/10/2021 to 07/19/2021


Fetching NVDA news: 25date(s) [00:02,  9.75date(s)/s]


Total unique dates collected: 25
Collected 55 rows from 07/01/2021 to 07/10/2021


In [ ]:
df_all.tail()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
19280,https://seekingalpha.com/article/4437455-nvidi...,https://cdn.snapi.dev/images/v1/e/r/x/semi10-8...,"Is Nvidia A Good Dividend Stock? Yes, But Not Now",Nvidia Corp pays a paltry but growing dividend...,Seeking Alpha,"Thu, 01 Jul 2021 11:05:00 -0400","[dividend, paylimitwall]",Neutral,Article,[NVDA]
19281,https://www.marketwatch.com/story/nvidia-price...,https://cdn.snapi.dev/images/v1/h/i/x/semi7-89...,Nvidia price target raised to new Street high ...,BMO Capital Markets analyst Ambrish Srivastava...,Market Watch,"Thu, 01 Jul 2021 09:18:22 -0400",[paylimitwall],Positive,Article,[NVDA]
19282,https://www.fool.com/investing/2021/07/01/is-n...,https://cdn.snapi.dev/images/v1/k/o/s/semi8-89...,Is NVIDIA Headed to $900? 3 Things to Know,The growth opportunities are mounting for this...,The Motley Fool,"Thu, 01 Jul 2021 09:15:00 -0400",[],Positive,Article,[NVDA]
19283,https://www.zacks.com/stock/news/1756514/5-soa...,https://cdn.snapi.dev/images/v1/x/j/5/computer...,5 Soaring Tech Growth Stocks to Buy Instead of...,Here we pick five technology stocks - Transcat...,Zacks Investment Research,"Thu, 01 Jul 2021 09:01:11 -0400",[],Positive,Article,"[DDD, GNRC, NVDA, PERI, TRNS]"
19284,https://www.fool.com/investing/2021/07/01/buy-...,https://cdn.snapi.dev/images/v1/0/s/i/semi29-8...,Buy This Stock Before It Turns Into a Top 5G Play,This tech stock is already enjoying terrific g...,The Motley Fool,"Thu, 01 Jul 2021 07:16:00 -0400",[],Positive,Article,[NVDA]


In [ ]:
# data_last_week = fetch_news_from_2021_07_20(
#             start_date="07202021",
#             end_date="07272021",
#             ticker = "TSM"
#         )

# df_last_week = pd.DataFrame(data_last_week)
# full_df = pd.concat([df_all, df_last_week], ignore_index=True)
# print(len(full_df))

In [ ]:
full_df = df_all.copy()

In [ ]:
full_df.to_csv(f"{ticker1}_articles_07102021_10052025.csv")

In [ ]:
# full_df = pd.read_csv("/content/drive/MyDrive/SI699 Files/NVDA_articles_07202021_10052025.csv")

In [ ]:
len(full_df)

19285

In [ ]:
full_df.head()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
0,https://www.fool.com/investing/2025/10/05/thes...,https://cdn.snapi.dev/images/v1/w/r/l/semi18-3...,These 2 AI Stocks Are Money-Printing Machines,The AI market is booming. Bain projects the to...,The Motley Fool,"Sun, 05 Oct 2025 18:26:00 -0400",[],Positive,Article,"[AVGO, NVDA]"
1,https://www.fool.com/investing/2025/10/05/this...,https://cdn.snapi.dev/images/v1/q/q/n/nvda25-2...,Prediction: This Artificial Intelligence (AI) ...,Cars are continually becoming smarter as autom...,The Motley Fool,"Sun, 05 Oct 2025 07:39:00 -0400",[],Positive,Article,[NVDA]
2,https://www.fool.com/investing/2025/10/05/nvid...,https://cdn.snapi.dev/images/v1/f/k/4/software...,Nvidia vs. Microsoft: Which Stock Is the Bette...,Nvidia (NVDA -0.77%) and Microsoft (MSFT 0.26%...,The Motley Fool,"Sun, 05 Oct 2025 07:15:00 -0400",[],Positive,Article,"[MSFT, NVDA]"
3,https://www.fool.com/investing/2025/10/05/core...,https://cdn.snapi.dev/images/v1/d/t/w/podc11-3...,CoreWeave's $6.3 Billion Backstop Deal With Nv...,As the artificial intelligence (AI) data cente...,The Motley Fool,"Sun, 05 Oct 2025 06:30:00 -0400",[podcast],Negative,Article,[NVDA]
4,https://www.fool.com/investing/2025/10/05/is-c...,https://cdn.snapi.dev/images/v1/o/j/e/software...,Is CoreWeave a Better Investment Than Nvidia?,CoreWeave (CRWV -2.33%) is making some big mov...,The Motley Fool,"Sun, 05 Oct 2025 05:15:00 -0400",[],Neutral,Article,"[CRWV, NVDA]"


In [ ]:
full_df.tail()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
19280,https://seekingalpha.com/article/4437455-nvidi...,https://cdn.snapi.dev/images/v1/e/r/x/semi10-8...,"Is Nvidia A Good Dividend Stock? Yes, But Not Now",Nvidia Corp pays a paltry but growing dividend...,Seeking Alpha,"Thu, 01 Jul 2021 11:05:00 -0400","[dividend, paylimitwall]",Neutral,Article,[NVDA]
19281,https://www.marketwatch.com/story/nvidia-price...,https://cdn.snapi.dev/images/v1/h/i/x/semi7-89...,Nvidia price target raised to new Street high ...,BMO Capital Markets analyst Ambrish Srivastava...,Market Watch,"Thu, 01 Jul 2021 09:18:22 -0400",[paylimitwall],Positive,Article,[NVDA]
19282,https://www.fool.com/investing/2021/07/01/is-n...,https://cdn.snapi.dev/images/v1/k/o/s/semi8-89...,Is NVIDIA Headed to $900? 3 Things to Know,The growth opportunities are mounting for this...,The Motley Fool,"Thu, 01 Jul 2021 09:15:00 -0400",[],Positive,Article,[NVDA]
19283,https://www.zacks.com/stock/news/1756514/5-soa...,https://cdn.snapi.dev/images/v1/x/j/5/computer...,5 Soaring Tech Growth Stocks to Buy Instead of...,Here we pick five technology stocks - Transcat...,Zacks Investment Research,"Thu, 01 Jul 2021 09:01:11 -0400",[],Positive,Article,"[DDD, GNRC, NVDA, PERI, TRNS]"
19284,https://www.fool.com/investing/2021/07/01/buy-...,https://cdn.snapi.dev/images/v1/0/s/i/semi29-8...,Buy This Stock Before It Turns Into a Top 5G Play,This tech stock is already enjoying terrific g...,The Motley Fool,"Thu, 01 Jul 2021 07:16:00 -0400",[],Positive,Article,[NVDA]


In [ ]:
full_df["parsed_date"] = pd.to_datetime(
    full_df["date"],
    format="%a, %d %b %Y %H:%M:%S %z",  # <-- strict format to ensure datetime
    errors="coerce"
)

/tmp/ipython-input-3243511048.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  full_df["parsed_date"] = pd.to_datetime(


In [ ]:
full_df['parsed_date'].value_counts()

,count
parsed_date,
2025-06-11 07:00:00-04:00,5
2023-03-21 12:15:00-04:00,5
2024-11-24 06:15:00-05:00,4
2025-03-30 07:15:00-04:00,4
2022-05-30 12:30:00-04:00,4
...,...
2024-10-21 08:53:17-04:00,1
2024-10-21 09:19:57-04:00,1
2024-10-21 09:31:22-04:00,1


In [ ]:
full_df["parsed_date"] = full_df["parsed_date"].astype(str).str.strip()

# 2) parse to a single tz (UTC) so dtype becomes datetime64[ns, UTC]
full_df["parsed_date"] = pd.to_datetime(full_df["parsed_date"], errors="coerce", utc=True)

print(full_df["parsed_date"].dtype)  # should show datetime64[ns, UTC]

# 3) if you want America/Detroit, convert
full_df["parsed_date_local"] = full_df["parsed_date"].dt.tz_convert("America/Detroit")

# 4) build date_only and time_only from whichever you prefer
full_df["date_only"] = full_df["parsed_date_local"].dt.strftime("%Y-%m-%d")
full_df["time_only"] = full_df["parsed_date_local"].dt.strftime("%H:%M:%S")

datetime64[ns, UTC]


In [ ]:
full_df

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,parsed_date,parsed_date_local,date_only,time_only
0,https://www.fool.com/investing/2025/10/05/thes...,https://cdn.snapi.dev/images/v1/w/r/l/semi18-3...,These 2 AI Stocks Are Money-Printing Machines,The AI market is booming. Bain projects the to...,The Motley Fool,"Sun, 05 Oct 2025 18:26:00 -0400",[],Positive,Article,"[AVGO, NVDA]",2025-10-05 22:26:00+00:00,2025-10-05 18:26:00-04:00,2025-10-05,18:26:00
1,https://www.fool.com/investing/2025/10/05/this...,https://cdn.snapi.dev/images/v1/q/q/n/nvda25-2...,Prediction: This Artificial Intelligence (AI) ...,Cars are continually becoming smarter as autom...,The Motley Fool,"Sun, 05 Oct 2025 07:39:00 -0400",[],Positive,Article,[NVDA],2025-10-05 11:39:00+00:00,2025-10-05 07:39:00-04:00,2025-10-05,07:39:00
2,https://www.fool.com/investing/2025/10/05/nvid...,https://cdn.snapi.dev/images/v1/f/k/4/software...,Nvidia vs. Microsoft: Which Stock Is the Bette...,Nvidia (NVDA -0.77%) and Microsoft (MSFT 0.26%...,The Motley Fool,"Sun, 05 Oct 2025 07:15:00 -0400",[],Positive,Article,"[MSFT, NVDA]",2025-10-05 11:15:00+00:00,2025-10-05 07:15:00-04:00,2025-10-05,07:15:00
3,https://www.fool.com/investing/2025/10/05/core...,https://cdn.snapi.dev/images/v1/d/t/w/podc11-3...,CoreWeave's $6.3 Billion Backstop Deal With Nv...,As the artificial intelligence (AI) data cente...,The Motley Fool,"Sun, 05 Oct 2025 06:30:00 -0400",[podcast],Negative,Article,[NVDA],2025-10-05 10:30:00+00:00,2025-10-05 06:30:00-04:00,2025-10-05,06:30:00
4,https://www.fool.com/investing/2025/10/05/is-c...,https://cdn.snapi.dev/images/v1/o/j/e/software...,Is CoreWeave a Better Investment Than Nvidia?,CoreWeave (CRWV -2.33%) is making some big mov...,The Motley Fool,"Sun, 05 Oct 2025 05:15:00 -0400",[],Neutral,Article,"[CRWV, NVDA]",2025-10-05 09:15:00+00:00,2025-10-05 05:15:00-04:00,2025-10-05,05:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19280,https://seekingalpha.com/article/4437455-nvidi...,https://cdn.snapi.dev/images/v1/e/r/x/semi10-8...,"Is Nvidia A Good Dividend Stock? Yes, But Not Now",Nvidia Corp pays a paltry but growing dividend...,Seeking Alpha,"Thu, 01 Jul 2021 11:05:00 -0400","[dividend, paylimitwall]",Neutral,Article,[NVDA],2021-07-01 15:05:00+00:00,2021-07-01 11:05:00-04:00,2021-07-01,11:05:00
19281,https://www.marketwatch.com/story/nvidia-price...,https://cdn.snapi.dev/images/v1/h/i/x/semi7-89...,Nvidia price target raised to new Street high ...,BMO Capital Markets analyst Ambrish Srivastava...,Market Watch,"Thu, 01 Jul 2021 09:18:22 -0400",[paylimitwall],Positive,Article,[NVDA],2021-07-01 13:18:22+00:00,2021-07-01 09:18:22-04:00,2021-07-01,09:18:22
19282,https://www.fool.com/investing/2021/07/01/is-n...,https://cdn.snapi.dev/images/v1/k/o/s/semi8-89...,Is NVIDIA Headed to $900? 3 Things to Know,The growth opportunities are mounting for this...,The Motley Fool,"Thu, 01 Jul 2021 09:15:00 -0400",[],Positive,Article,[NVDA],2021-07-01 13:15:00+00:00,2021-07-01 09:15:00-04:00,2021-07-01,09:15:00
19283,https://www.zacks.com/stock/news/1756514/5-soa...,https://cdn.snapi.dev/images/v1/x/j/5/computer...,5 Soaring Tech Growth Stocks to Buy Instead of...,Here we pick five technology stocks - Transcat...,Zacks Investment Research,"Thu, 01 Jul 2021 09:01:11 -0400",[],Positive,Article,"[DDD, GNRC, NVDA, PERI, TRNS]",2021-07-01 13:01:11+00:00,2021-07-01 09:01:11-04:00,2021-07-01,09:01:11


In [ ]:
full_df['date_only'].nunique()

1421

In [ ]:
len(full_df)

19285

In [ ]:
full_df.to_csv(f"/content/drive/MyDrive/SI699 Files/{ticker1}_articles_07102021_10052025.csv")

In [ ]:
full_df.to_csv("nvda_articles_19k.csv")

In [ ]:
# nvda = pd.read_csv("/content/drive/MyDrive/SI699 Files/NVDA_articles_07202021_10052025.csv")